# Mật mã bất đối xứng
Mật mã khóa công khai, hay mật mã bất đối xứng, là một hệ thống mật mã sử dụng các cặp khóa gồm: 
+ Khóa công khai, có thể được phổ biến rộng rãi mà không ảnh hưởng đến bảo mật.
+ Khóa riêng tư, chỉ được biết bởi chủ sở hữu. 

Trong một hệ thống, bất kỳ người nào cũng có thể mã hóa tin nhắn bằng khóa công khai, nhưng tin nhắn được mã hóa đó chỉ có thể được giải mã bằng khóa riêng của người nhận.

Ngoài ra, người gửi có thể đính tin nhắn với khóa riêng tư để tạo chữ ký số trên tin nhắn. Bất kỳ ai có khóa công khai có tin nhắn trên và chữ ký số tương ứng để xác minh xem chữ ký có hợp lệ hay không, tức là do chủ sở hữu của khóa riêng tư tương ứng tạo.

# Thuật toán RSA

RSA (Rivest–Shamir–Adleman) là một trong những hệ thống mật mã khóa công khai đầu tiên và được sử dụng rộng rãi trong việc truyền dữ liệu an toàn. Từ viết tắt RSA là các chữ cái đầu trong họ của Ron Rivest, Adi Shamir và Leonard Adleman, những người đã mô tả công khai thuật toán này vào năm 1977. Trong đó, trong 1 hệ thống mật mã, khóa mã hóa là khóa công khai và khóa giải mã được giữ bí mật (riêng tư).

Người dùng thuật toán RSA dựa trên hai số nguyên tố lớn và giá trị phụ trợ mà tạo khóa công khai và sau đó công khai khóa. Trong đó, các số nguyên tố phải được giữ bí mật. Bất kỳ ai cũng có thể sử dụng khóa công khai để mã hóa tin nhắn, nhưng chỉ người có biết về cặp số nguyên tố mới có thể giải mã tin nhắn. Tuy nhiên RSA vẫn có thể bị phá và được gọi là sự cố RSA. Nhưng ở thời điểm hiện tại, không có phương pháp nào có thể đánh bại hệ thống nếu hệ thống sử dụng khóa đủ lớn.

## 1. Tiền xử lý: 
## 1.1. Thư viện sử dụng:

In [ ]:
import random
import math

## 1.2. Chuyển đổi: chuỗi ký tự  <=> dãy số thập phân 

In [ ]:
# Hàm chuyển đổi 1 chuỗi ký tự  <=> 1 dãy số nguyên thập phân. Mục đích chuẩn hóa các chuỗi ký tự sang dãy số nguyên để mã hóa.
# Input 1: (String) decStr: Chuỗi ký tự OR danh sách thập phân 
# Input 2: (int) flag: 0 = cờ chuyển sang danh sách thập phân, 1 = cờ chuyển sang chuỗi
# Output: (list) Danh sách thập phân OR (String) Chuỗi ký tự
def convertDecAndStr(decStr, flag):
    decStr_list = []
    if flag == 0:
      for i in decStr: decStr_list.append(ord(i)) # Duyệt từng ký tự trong chuỗi ký tự chuyển sang từng số nguyên
      return decStr_list
    else:
      for i in decStr: decStr_list.append(chr(i)) # Duyệt từng số nguyên trong danh sách số nguyên chuyển sang từng ký tự
      return ''.join(decStr_list)

# Hàm tìm ước chung lớn nhất 2 số a,b
# Input: (Int) a: số nguyên a
# Input: (Int) b: số nguyên b
# Output:(Int) ước chung lớn nhất
def gcd(a, b):
    while a:
        b, a = a, b % a
    return b

#---------------------------------------------------------------------------------
print('Ví dụ: ')
mes = '@12334Test a ba la trap'
print('Văn bản gốc:\n\t', mes)

dec = convertDecAndStr(mes, 0)
print('Văn bản gốc => chuỗi thập phân:\n\t', dec)

txt = convertDecAndStr(dec, 1)
print('Chuỗi thập phân => Văn bản gốc:\n\t', txt)

Ví dụ: 
Văn bản gốc:
	 @12334Test a ba la trap
Văn bản gốc => chuỗi thập phân:
	 [64, 49, 50, 51, 51, 52, 84, 101, 115, 116, 32, 97, 32, 98, 97, 32, 108, 97, 32, 116, 114, 97, 112]
Chuỗi thập phân => Văn bản gốc:
	 @12334Test a ba la trap


## 1.3. Khởi tạo số nguyên tố LỚN
Ý tưởng về RSA được dựa trên 1 thực tế là chúng ta rất khó để phân tích một số nguyên lớn. Khóa công khai bao gồm hai số trong đó có một số là phép nhân của hai số nguyên tố lớn. Và khóa riêng tư cũng được lấy từ hai số nguyên tố giống nhau. Vì vậy, nếu ai đó có thể phân tích được số lớn đó, thì khóa riêng tư sẽ bị xâm phạm. Do đó, độ mạnh của mã hóa hoàn toàn phụ thuộc vào kích thước khóa và nếu chúng ta tăng gấp đôi hoặc gấp ba kích thước khóa, độ mạnh của mã hóa sẽ tăng theo cấp số nhân. Các khóa RSA thường có độ dài 1024 hoặc 2048 bit, nhưng các chuyên gia tin rằng các khóa 1024 bit có thể bị phá vỡ trong tương lai gần. Tuy nhiên cho đến nay nó dường như là một nhiệm vụ bất khả thi.

Với sàng của Eratosthenes và cách kiểm tra tính nguyên tố Rabin Miller. Ta sẽ khởi tạo số nguyên tố ngẫu nhiên trong giá trị n bit nhất định:

In [ ]:
# Nguồn: https://www.geeksforgeeks.org/how-to-generate-large-prime-numbers-for-rsa-algorithm/
# Danh sách các số nguyên tố tạo trước để kiểm
first_primes_list = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
                     31, 37, 41, 43, 47, 53, 59, 61, 67,
                     71, 73, 79, 83, 89, 97, 101, 103,
                     107, 109, 113, 127, 131, 137, 139,
                     149, 151, 157, 163, 167, 173, 179,
                     181, 191, 193, 197, 199, 211, 223,
                     227, 229, 233, 239, 241, 251, 257,
                     263, 269, 271, 277, 281, 283, 293,
                     307, 311, 313, 317, 331, 337, 347, 349]
 
 # Tạo 1 số lẻ có n bit trong khoảng 2**(n-1)+1 and 2**n-1 
 # Do tất cả các số nguyên tố (> 2) đều là số lẻ nên để có hiệu suất tốt hơn, chỉ có thể chọn số lẻ
 # Input: (Int) n = số bit
 # Output: (Int) 1 số lẻ ngẫu nhiên n bit
def nBitRandom(n):
    return random.randrange(2**(n-1)+1, 2**n - 1)
 
 # Tạo một số chia hết bởi số nguyên tố đầu tiên. 
 # Bước này là một bài kiểm tra tính nguyên tố ở mức độ thấp.
 # Yêu cầu tính toán trước vài trăm số nguyên tố đầu tiên (sử dụng sàng của Eratosthenes)
 # 1 số được chia cho các số nguyên tố (được tạo trước) để kiểm tra tính chia hết. 
 # Nếu chia hết, phép thử không thành công và một số mới phải được chọn và kiểm tra. 
 # Bước trên được lặp đi lặp lại miễn là tìm thấy một giá trị nguyên tố cùng nhau 
 # với tất cả các số nguyên tố trong danh sách số nguyên tố đã tạo
 # Input: (Int) n = số bit
 # Output: (Int) số nguyên tố cùng nhau (chỉ là kiểm bước 1)
def getLowLevelPrime(n):
    '''Generate a prime candidate divisible
    by first primes'''
    while True:
        # Lấy số ngẫu nhiên
        pc = nBitRandom(n)
 
        # Kiểm tra tính chia hết của các số nguyên tố đã cho trước
        for divisor in first_primes_list:
            if pc % divisor == 0 and divisor**2 <= pc:
                break
        # Nếu không tìm thấy số chia, trả về giá trị
        else: return pc
 
 # Khi đã vượt qua hàm kiểm tra cấp thấp getLowLevelPrime(), thì sẽ được kiểm tra lại bằng Bài kiểm tra tính nguyên tố Rabin Miller.
 # Nếu một số sau khi kiểm tra ở cấp thấp, khi qua một lần lặp lại của bài kiểm tra Rabin Miller, thì xác suất của số đó là số nguyên tố là 75%.
 # Do đó, nếu số vượt qua bài kiểm tra với đủ số lần, có thể được coi là số nguyên tố với mức xác suất thỏa mãn.
 # Input:(Int) số nguyên tố cần kiểm tra
 # Output: (Bool) True = là số nguyên tố đúng, False = không là số nguyên tố
def isMillerRabinPassed(mrc):
    '''Run 20 iterations of Rabin Miller Primality test'''
    maxDivisionsByTwo = 0
    ec = mrc-1
    while ec % 2 == 0:
        ec >>= 1
        maxDivisionsByTwo += 1
    assert(2**maxDivisionsByTwo * ec == mrc-1)
 
    def trialComposite(round_tester):
        if pow(round_tester, ec, mrc) == 1:
            return False
        for i in range(maxDivisionsByTwo):
            if pow(round_tester, 2**i * ec, mrc) == mrc-1:
                return False
        return True
 
    # Số kiểm tra thử ở đây. Chạy 20 lần lặp lại bài kiểm tra số nguyên tố của Rabin Miller
    numberOfRabinTrials = 20
    for i in range(numberOfRabinTrials):
        round_tester = random.randrange(2, mrc)
        if trialComposite(round_tester):
            return False
    return True
 
 # Hàm phát sinh số nguyên tố
# Input 1: (Int) n: số bit để khởi tạo
# Output: (Int) prime_candidate: số nguyên tố
def generatePrime(n=12):
    while True:
        prime_candidate = getLowLevelPrime(n)
        if not isMillerRabinPassed(prime_candidate):
            continue
        else: return prime_candidate

#---------------------------------------------------------------------------------
print('Ví dụ: ')
print('Số nguyên tố được tạo với 12 bit:', generatePrime())

Ví dụ: 
Số nguyên tố được tạo với 12 bit: 2789


## 2. Khởi tạo khóa công khai và khóa riêng tư

#### **Khóa công khai tương ứng với cặp {e, n}**

Trong đó, $e$ phải: 
* $1 < e < \phi(n)$
* nguyên tố cùng nhau với $n$ và $\phi(n)$

$\phi(n) = (p - 1) * (q - 1)$ = số lượng nguyên tố cùng nhau với n

#### **Khóa riêng tư tương ứng với cặp {d, n}**

Trong đó, $d$ phải: 
* $d e (mod \phi(n)) = 1$

In [ ]:
# Hàm phát sinh n = p * q
# Input 1: (Int) p: số p nguyên tố bất kỳ
# Input 2: (Int) q: số q nguyên tố bất kỳ
# Output: (Int) khóa n
def generateN(p, q):
    return p * q

# Hàm phát sinh phi
# Input 1: (Int) p: số p nguyên tố bất kỳ
# Input 2: (Int) q: số q nguyên tố bất kỳ
# Output: (Int) số phi
def generatePhi(p, q):
    return (p-1) * (q-1)

# Khóa E 
# Hàm phát sinh e (e với n  là khóa công khai)
# Input 1: (Int) p: số p nguyên tố bất kỳ
# Input 2: (Int) q: số q nguyên tố bất kỳ
# Output: (Int) e = khóa e
def generateE(p, q):
    phi = generatePhi(p, q)

    for e in range(random.randrange(3, phi-1, 2), phi-1):
        if gcd(e, phi) == 1: return e

# Hàm phát sinh D (d với n này được coi là khóa riêng tư)
# Input : (Int) e: số e (Khóa E)
# Output: (Int) d: khóa d
def generateD(e):
    phi = generatePhi(p, q)

    d = int(phi / e)
    while (True):
        if (d * e) % phi == 1: return d
        d += 1

#---------------------------------------------------------------------------------
print('Ví dụ: ')
# Tạo 2 số p, q nguyên tố ngẫu nhiên
p = generatePrime()
q = generatePrime()

# Nếu p mà trùng q, thì ta phải tạo lại q. Vì p và q phải khác nhau.
while (p == q):
    q = generatePrime()

print('Hai số nguyên tố ngẫu nhiên:')
print('\tSố nguyên tố thứ 1: ', p)
print('\tSố nguyên tố thứ 2: ', q)

n = generateN(p, q)
e = generateE(p, q)
d = generateD(e)

print('\nKhóa n: ', n)
print('khóa E: ', e)
print('Khóa D: ', d)

print('\nKhóa công khai (e, n): (%d, %d)' %(e, n))
print('Khóa riêng tư (d, n): (%d, %d)' %(d, n))

Ví dụ: 
Hai số nguyên tố ngẫu nhiên:
	Số nguyên tố thứ 1:  3457
	Số nguyên tố thứ 2:  2281

Khóa n:  7885417
khóa E:  7062107
Khóa D:  362963

Khóa công khai (e, n): (7062107, 7885417)
Khóa riêng tư (d, n): (362963, 7885417)


## 3. Thuật toán mã hóa và giải mã

Công thức để mã hóa và giải mã các tin nhắn là như nhau. Tất cả các tin nhắn được mã hóa bằng khóa công khai chỉ có thể được giải mã bằng khóa riêng tư và các tin nhắn được mã hóa bằng khóa riêng tư chỉ có thể được giải mã bằng khóa công khai.

In [ ]:
decrypted = [encryptAndDecrypt(i, dKey, nKey) for i in encrypted]

# Hàm mã hóa và giải mã
# Input 1: (Int) des: số dạng thập phân được mã hóa từ 1 thành phần tin
# Input 2: (Int) edKey: khóa e hoặc d
# Input 3: (Int) n: số n trong cặp khóa
# Output: (Int) res: số dạng thập phân đã được giải mã
def encryptAndDecrypt(des, edKey, n):
    res = 1  
    des = des % n  
      
    if (des == 0) : return 0
  
    while (edKey > 0) :          
        # Nếu khóa là số lẻ, nhân khóa với kết quả
        if ((edKey & 1) == 1) : res = (res * des) % n 
  
        # khóa phải là chẳn
        edKey = edKey >> 1
        des = (des * des) % n        
    return res

## 4.1. Ví dụ: mã hóa bằng khóa công khai và giải mã bằng khóa riêng tư
Sau khi tin nhắn được mã hóa bằng khóa công khai, nó chỉ có thể được giải mã bởi chủ sở hữu khóa riêng tư. Và theo cách này, tin nhắn không thể bị sửa đổi trong quá trình truyền đi. 

Phát sinh 2 số nguyên tố p,q và cặp khóa công khai:

In [87]:
# Khởi tạo 2 số p, q nguyên tố ngẫu nhiên.
p = generatePrime()
q = generatePrime()

# Nếu p mà trùng q, thì ta phải tạo lại q, cho đến khi p,q phải khác nhau.
while (p == q):
    q = generatePrime()

# Khởi tạo n và cặp khóa e,d 
n = generateN(p, q)
e = generateE(p, q)

print('Hai số nguyên tố ngẫu nhiên:')
print('\tSố nguyên tố p: ', p)
print('\tSố nguyên tố q: ', q)

print('\nKhóa n: ', n)
print('khóa E: ', e)
print('\nKhóa công khai (e, n):')
print('(%d, %d)' %(e, n))

Hai số nguyên tố ngẫu nhiên:
	Số nguyên tố p:  2447
	Số nguyên tố q:  2347

Khóa n:  5743109
khóa E:  2412421

Khóa công khai (e, n):
(2412421, 5743109)


Nhập tin nhắn cần mã hóa và sao chép - dán khóa công khai đã được tạo phía trên, để khởi tạo tin nhắn mã hóa và cặp khóa riêng tư.

In [89]:
#@title String fields
Tin_Nhan_Can_Ma_Hoa = 'con c\xE1 b\u1ED1ng' #@param {type:"string"}
Cap_Khoa_Cong_Khai = '(2412421, 5743109)' #@param {type:"string"}

# Chuyển chuỗi ký tự đoạn mes sang danh sách dãy thập phân
mesDec = convertDecAndStr(Tin_Nhan_Can_Ma_Hoa, 0)

# Hàm định dạng lại input khóa
# Input 1: (Int) txtKeys: input khóa
# Output: (Int) listKeys: khóa đã được định dạng
def formatPubKey(txtKeys):
  listKeys = txtKeys.replace('(', '').replace(')', '').split(", ")
  listKeys = list(map(int, listKeys))
  return listKeys

eKey = formatPubKey(Cap_Khoa_Cong_Khai)[0]
nKey = formatPubKey(Cap_Khoa_Cong_Khai)[1]

# Mã hóa tin nhắn bằng khóa công khai (e)
encrypted = [encryptAndDecrypt(i, eKey, nKey) for i in mesDec]

d = generateD(eKey)

print('Tin nhắn:\n\t', Tin_Nhan_Can_Ma_Hoa)
print('Tin nhắn theo chuỗi thập phân:\n\t', mesDec)
print('\nTin nhắn đã được mã hóa:')
print(encrypted)
print('\nKhóa n: ', n)
print('Khóa D: ', d)
print('Khóa riêng tư (d, n):')
print('(%d, %d)' %(d, n))

Tin nhắn:
	 con cá bống
Tin nhắn theo chuỗi thập phân:
	 [99, 111, 110, 32, 99, 225, 32, 98, 7889, 110, 103]

Tin nhắn đã được mã hóa:
[680440, 4002111, 2172975, 813200, 680440, 4448182, 813200, 2873186, 2542723, 2172975, 1070449]

Khóa n:  5743109
Khóa D:  3618697
Khóa riêng tư (d, n):
(3618697, 5743109)


Nhập tin nhắn đã mã hóa (chuỗi nhị phân) và sao chép - dán khóa riêng tư đã được tạo phía trên, để giải mã tin nhắn.

In [90]:
#@title String fields
Tin_Nhan_Can_Giai_Ma = '[680440, 4002111, 2172975, 813200, 680440, 4448182, 813200, 2873186, 2542723, 2172975, 1070449]' #@param {type:"string"}
Cap_Khoa_Rieng_Tu = '(3618697, 5743109)' #@param {type:"string"}

# Hàm định dạng lại input chuỗi thập phân tin nhắn mã hóa
# Input 1: (Int) strList: input chuỗi thập phân tin nhắn mã hóa
# Output: (Int) intLs: chuỗi thập phân tin nhắn mã hóa đã được định dạng
def formatStringList(strList):
  ls = strList.strip('][').split(', ')
  intLs = list(map(int, ls))
  return intLs

dKey = formatPubKey(Cap_Khoa_Rieng_Tu)[0]
nKey = formatPubKey(Cap_Khoa_Rieng_Tu)[1]
encryptedStr = formatStringList(Tin_Nhan_Can_Giai_Ma)

# Giải mã tin nhắn bằng khóa riêng tư (d)
decrypted = [encryptAndDecrypt(i, dKey, nKey) for i in encrypted]
print(decrypted)
print('\n\nGiải mã tin nhắn:\n\t', convertDecAndStr(decrypted, 1))
print('Tin nhắn theo chuỗi thập phân sau khi giải mã:\n\t', decrypted)


[99, 111, 110, 32, 99, 225, 32, 98, 7889, 110, 103]


Giải mã tin nhắn:
	 con cá bống
Tin nhắn theo chuỗi thập phân sau khi giải mã:
	 [99, 111, 110, 32, 99, 225, 32, 98, 7889, 110, 103]


## 4.2. Ví dụ: Mã hóa bằng khóa riêng tư và giải mã bằng khóa công khai
Việc mã hóa tin nhắn bằng khóa riêng tư này lại không thực sự có ý nghĩa, vì khóa công khai được mọi người biết đến, bất kỳ ai cũng có thể giải mã được. tuy nhiên, mã hóa bằng khóa công khai được sử dụng theo cách khác là để chứng minh rằng tin nhắn thực sự đến từ chủ sở hữu của khóa riêng tư. Đó chính là ứng dụng cho chữ ký số. Người chính chủ dùng khoá riêng tư để ký tên và các người khác dùng khoá công khai để xác minh chữ ký đó có phải chính chủ không.

In [ ]:
sig = 'Tôi là Thành Cry'
sigDec = convertDecAndStr(sig, 0)

# Chuyển chuỗi ký tự đoạn khóa sang danh sách dãy thập phân
mesDec = convertDecAndStr(mes, 0)

# Khởi tạo 2 số p, q nguyên tố ngẫu nhiên.
p = generatePrime()
q = generatePrime()

# Nếu p mà trùng q, thì ta phải tạo lại q, cho đến khi p,q phải khác nhau.
while (p == q):
    q = generatePrime()

# Khởi tạo n và cặp khóa e,d 
n = generateN(p, q)
e = generateE(p, q)
d = generateD(e)

# Mã hóa chữ ký bằng khóa riêng tư (D)
encrypted = [encryptAndDecrypt(i, d, n) for i in sigDec]

# Giải mã chữ ký bằng khóa công khai (E)
decrypted = [encryptAndDecrypt(i, e, n) for i in encrypted]

print('Hai số nguyên tố ngẫu nhiên:')
print('\tSố nguyên tố p: ', p)
print('\tSố nguyên tố q: ', q)

print('\nKhóa n: ', n)
print('khóa E: ', e)
print('Khóa D: ', d)

print('\nKhóa công khai {e, n}: {%d, %d}' %(e, n))
print('Khóa riêng tư {d, n}: {%d, %d}' %(d, n))

print('Chữ ký:\n\t', sig)
print('Chữ ký theo chuỗi thập phân:\n\t', sigDec)
print('\nMã hóa chữ ký:\n\t', encrypted)
print('\n\nGiải mã chữ ký:\n\t', convertDecAndStr(decrypted, 1))
print('Chữ ký theo chuỗi thập phân sau khi giải mã:\n\t', decrypted)


Hai số nguyên tố ngẫu nhiên:
	Số nguyên tố p:  2237
	Số nguyên tố q:  3947

Khóa n:  8829439
khóa E:  4370989
Khóa D:  5435533

Khóa công khai {e, n}: {4370989, 8829439}
Khóa riêng tư {d, n}: {5435533, 8829439}
Chữ ký:
	 Tôi là Thành Cry
Chữ ký theo chuỗi thập phân:
	 [84, 244, 105, 32, 108, 224, 32, 84, 104, 224, 110, 104, 32, 67, 114, 121]

Mã hóa chữ ký:
	 [7762543, 3880204, 5326976, 1133201, 4265264, 546198, 1133201, 7762543, 8733341, 546198, 6538639, 8733341, 1133201, 1444799, 94863, 2522262]


Giải mã chữ ký:
	 Tôi là Thành Cry
Chữ ký theo chuỗi thập phân sau khi giải mã:
	 [84, 244, 105, 32, 108, 224, 32, 84, 104, 224, 110, 104, 32, 67, 114, 121]
